In [1]:
from ast import If
import json
import csv
import os
import glob
import numpy as np
import playerData

jsonFiles = []
csvFiles = []
playersArray = []
numNiveles = 4
nivel = -1
muertesPorNivel = np.zeros(numNiveles) #esto mide la suma de todas las muertes hechas por todos los jugadores?¿
saltosPorNivel = np.zeros(numNiveles)  #esto mide la suma de todos los saltos hechos por todos los jugadores?¿
numArchivos = 0


def ProcessJson(f):
    Lines = f.readlines()
    count = 0
    # For each line process one event
    events = []
    for line in Lines:
        count += 1
        events.append(json.loads(line))
    return events

def ProcessCSV(f):

    players = []

    ncol = len(next(f))
    start = 3

    for row in f:
        answers = []
        name = str(row[start - 1])
        name = "".join([ch for ch in name if ch.isdigit()])   # getONLY the numbers in the name
        
        for x in range(start, ncol):
            answers.append(row[x])
        player = name, answers
        players.append(player)
    
    return players



# def Clean(f):
#     session_start = []
#     for i in range(0, f.readlines, 1):
#         line = f.readline[i]   

def ReadFiles():
    #List of files in folder
    for filename in glob.glob('**/*.json'):
        with open(filename, mode="r", encoding="utf-8") as f: # open files in readonly mode
            fName = f.name
            fName = "".join([ch for ch in fName if ch.isdigit()])   # getONLY the numbers in the name
            file = str(fName), ProcessJson(f)
            jsonFiles.append(file)
            player = playerData.Player(numNiveles)
            playersArray.append(player)
            global numArchivos
            numArchivos += 1
            
    for filename in glob.glob('*.csv'):
        with open(filename, mode="r", encoding="utf-8") as f:
            csvreader = csv.reader(f)
            file = str(f.name), ProcessCSV(csvreader)
            csvFiles.append(file)
            



A partir de ahora en la variable *jsonFiles* tenemos todos los archivos, y en cada archivo, los eventos separados en formato **json**

In [2]:
def SessionStart(e,player):
    ID = e["sesion"]
    player.setID(ID)
def SessionEnd(e,player):
    player.end()
def LevelStart(e,player):
    global nivel
    nivel = e["nivel"] -3 #Los niveles estan codificados como nivel 3, 4, 5 y 6, asi que les resto esto para que se guarden como 0,1,2, y 3.
    player.resetPath(nivel)

def LevelEnd(e):
    print("-------LEVELEND NOT IMPLEMENTED--------")

def Pause(e,player):
    player.pause()

def Unpause(e):
    print("-------UNPAUSE NOT IMPLEMENTED--------")

def Death(e, player):
    muertesPorNivel[nivel] += 1
    player.death(nivel)

def Collision(e):
    print("-------COLLISION NOT IMPLEMENTED--------")


def Jump(e,player):
    saltosPorNivel[nivel] += 1
    player.jump(nivel)

def PlayerPosition(e,player):
    pos = e["x"], e["y"]
    player.addToPath(pos, nivel)

In [3]:
def FormularioEnd(p):
        player = playersArray[p]
        player.formularioInforme()

def Age(a, player):
        player.setAge(a)

def GeneralExperience(a, player):
        player.setGeneralExperience(a)

def PlatformExperience(a, player):
        player.setPlatformExperience(a)

def Walljump(a, player):
        player.setWalljump(a)

def Gravity(a, player):
        player.setGravity(a)
        
def LevelDificulty(level, a, player):
        player.setLevelDificulty(level, a)

In [4]:
def ProcessEvent(e,actualPlayer):
    nombre = e["nombre"]
    player = playersArray[actualPlayer] #algunos eventos usaran al jugador y otros no

    if nombre == "SESSION_START":
         SessionStart(e,player)
        
    elif nombre == "SESSION_END": #descomentar para leer informes individuales
         SessionEnd(e,player)

    elif nombre == "LEVEL_START":
         LevelStart(e,player)

    # elif nombre == "LEVEL_END":
    #     LevelEnd(e)

    #elif nombre == "PAUSE":
    #     Pause(e,player)

    # elif nombre == "UNPAUSE":
    #     Unpause(e)

    elif nombre == "DEATH":
        Death(e,player)

    # elif nombre == "COLLISION":
    #     Collision(e)

    elif nombre == "JUMP":
        Jump(e,player)

    elif nombre == "PLAYER_POSITION":
         PlayerPosition(e,player)


    #print(nivel,":",nombre)




In [5]:
def ProcessAnswer(a, actualAnswer, actualPlayer):
    player = playersArray[actualPlayer]

    if(actualAnswer == 0):
        Age(a, player)

    elif(actualAnswer == 1):
        GeneralExperience(a, player)
        
    elif(actualAnswer == 2):
        PlatformExperience(a, player)
        
    elif(actualAnswer == 3):
        Walljump(a, player)
    
    elif(actualAnswer == 4):
        Gravity(a, player)
    
    elif(actualAnswer >= 5):
        LevelDificulty(actualAnswer - 5, a, player)
        

In [6]:

def ProcessFiles():
    ReadFiles()
    actualPlayer = -1 #iterador para añadir datos de jugador
    for f in jsonFiles:
        actualPlayer+=1
        print("Processing player", actualPlayer)
        for e in f[1]:
            ProcessEvent(e,actualPlayer)
    
    for f in csvFiles:
        for p in f[1]:
            index = 0
            found = False
            actualPlayer = 0

            for n in playersArray:
                if str(n.ID) == p[0]:
                    actualPlayer = index
                    found = True
                    break
                else: index += 1

            if found == True:
                print("Processing formulary", actualPlayer)
                actualAnswer= 0
                for a in p[1]:
                    ProcessAnswer(a, actualAnswer, actualPlayer)
                    actualAnswer += 1
                FormularioEnd(actualPlayer)
            else: print("Wrong ID: ", p[0], "\n---------------------------------------------")
                
            
            
    print("------------------------------------------")
    print("Media de saltos por jugador en cada nivel:",saltosPorNivel/np.size(playersArray))
    print("Media de muertes por jugador en cada nivel:",muertesPorNivel/np.size(playersArray))

ProcessFiles()

Processing player 0
INFORME JUGADOR  1030424204065505273
Jumps:  [11. 37. 17. 37.]
Deaths:  [0. 0. 0. 0.]
Tiempo en pausa:  0
---------------------------------------------
Processing player 1
INFORME JUGADOR  1060054125199479874
Jumps:  [ 33. 104.  27.  35.]
Deaths:  [0. 2. 0. 0.]
Tiempo en pausa:  0
---------------------------------------------
Processing player 2
INFORME JUGADOR  1454797195216275149
Jumps:  [ 20. 155.  25.  46.]
Deaths:  [0. 4. 0. 0.]
Tiempo en pausa:  0
---------------------------------------------
Processing player 3
INFORME JUGADOR  1482842417835807299
Jumps:  [195.  76.  54.   0.]
Deaths:  [6. 2. 0. 0.]
Tiempo en pausa:  0
---------------------------------------------
Processing player 4
INFORME JUGADOR  150124433152065388
Jumps:  [14. 36. 22. 35.]
Deaths:  [0. 0. 0. 0.]
Tiempo en pausa:  0
---------------------------------------------
Processing player 5
INFORME JUGADOR  1908151240582397456
Jumps:  [ 29. 120.  63.  45.]
Deaths:  [0. 1. 0. 0.]
Tiempo en pausa:  0